In [41]:
import SimpleITK as sitk
import pydicom
import numpy as np
import glob, os
from skimage.filters import gaussian

In [59]:
files = glob.glob(r'data/DigitalPhantomCT - Copy/*')
vof_location = (410, 247, 16)  # start, start, size
aif_location = (123, 251, 8)  # start, start, size
aif_values_rescaled_nosmoothing = [40.88909, 39.572624, 39.66895, 40.150585, 39.861603, 41.17807, 41.17807, 259.03726, 494.58847, 447.00302, 291.8205, 170.57712, 100.96492, 64.71393, 49.847485, 44.93482, 42.494537, 40.150585, 39.508408, 38.320374, 40.95331, 39.572624, 40.150585, 41.24229, 41.402832, 39.829494, 39.219425, 39.37997, 41.659706, 38.994663]
for ix, file in enumerate(files[::16]):
    ds = pydicom.dcmread(file)
    data = ds.pixel_array
    new_aif_value = aif_values_rescaled_nosmoothing[ix]
    new_aif_array = np.tile(np.array(new_aif_value), (8, 8))
    data[aif_location[0]:aif_location[0] + aif_location[2],
               aif_location[1]:aif_location[1] + aif_location[2]] = new_aif_array
    # copy the data back to the original data set
    ds.PixelData = data.tobytes()
    print('The information of the data set after downsampling: \n')
    print(ds.pixel_array[aif_location[0]:aif_location[0] + aif_location[2],
               aif_location[1]:aif_location[1] + aif_location[2]])
    ds.save_as(file)

The information of the data set after downsampling: 

[[40 40 40 40 40 40 40 40]
 [40 40 40 40 40 40 40 40]
 [40 40 40 40 40 40 40 40]
 [40 40 40 40 40 40 40 40]
 [40 40 40 40 40 40 40 40]
 [40 40 40 40 40 40 40 40]
 [40 40 40 40 40 40 40 40]
 [40 40 40 40 40 40 40 40]]
The information of the data set after downsampling: 

[[39 39 39 39 39 39 39 39]
 [39 39 39 39 39 39 39 39]
 [39 39 39 39 39 39 39 39]
 [39 39 39 39 39 39 39 39]
 [39 39 39 39 39 39 39 39]
 [39 39 39 39 39 39 39 39]
 [39 39 39 39 39 39 39 39]
 [39 39 39 39 39 39 39 39]]
The information of the data set after downsampling: 

[[39 39 39 39 39 39 39 39]
 [39 39 39 39 39 39 39 39]
 [39 39 39 39 39 39 39 39]
 [39 39 39 39 39 39 39 39]
 [39 39 39 39 39 39 39 39]
 [39 39 39 39 39 39 39 39]
 [39 39 39 39 39 39 39 39]
 [39 39 39 39 39 39 39 39]]
The information of the data set after downsampling: 

[[40 40 40 40 40 40 40 40]
 [40 40 40 40 40 40 40 40]
 [40 40 40 40 40 40 40 40]
 [40 40 40 40 40 40 40 40]
 [40 40 40 40 40 40 40 40

In [60]:
sd = 2
simulated_data_size = 32 * 7
scan_center = 512 // 2
simulated_data_start = scan_center - simulated_data_size // 2
simulated_data_end = scan_center + simulated_data_size // 2
other_files = sorted([file for file in files if file not in files[::16]])

for ix, file in enumerate(other_files):
    ds = pydicom.dcmread(file)
    data = ds.pixel_array
    phantomcurves = data[simulated_data_start:simulated_data_end,
                         simulated_data_start:simulated_data_end]
    truncate = np.ceil(2 * sd) / sd if sd != 0 else 0
    phantomcurves = gaussian(phantomcurves, sigma=(sd, sd), mode='nearest', truncate=truncate, preserve_range=True).astype(np.int16)
    data[simulated_data_start:simulated_data_end, simulated_data_start:simulated_data_end] = phantomcurves
    ds.PixelData = data.tobytes()
    ds.save_as(file)

In [16]:
    simulated_data_size = 32 * 7
    scan_center = image_data.shape[-1] // 2
    simulated_data_start = scan_center - simulated_data_size // 2
    simulated_data_end = scan_center + simulated_data_size // 2
    if gaussian_filter_type:
        perfusion_data = image_data[1:, :,
                         simulated_data_start:simulated_data_end,
                         simulated_data_start:simulated_data_end]
        perfusion_data = apply_gaussian_filter(gaussian_filter_type, perfusion_data.copy(), sd=sd)

The image has 512 x 512 voxels


In [19]:
data_downsampling = data
data_downsampling[:100, :100] = np.zeros((100,100))
print('The downsampled image has {} x {} voxels'.format(
    data_downsampling.shape[0], data_downsampling.shape[1]))

The downsampled image has 512 x 512 voxels


In [21]:
# copy the data back to the original data set
ds.PixelData = data_downsampling.tobytes()
# update the information regarding the shape of the data array
ds.Rows, ds.Columns = data_downsampling.shape

In [23]:
print('The information of the data set after downsampling: \n')
print(ds)
print(ds.pixel_array)
print(len(ds.PixelData))
ds.save_as("after.dcm")

The information of the data set after downsampling: 

Dataset.file_meta -------------------------------
(0002, 0000) File Meta Information Group Length  UL: 208
(0002, 0001) File Meta Information Version       OB: b'\x00\x01'
(0002, 0002) Media Storage SOP Class UID         UI: CT Image Storage
(0002, 0003) Media Storage SOP Instance UID      UI: 1.3.6.1.4.1.9590.100.1.1.167963314710984872700115334653369750841
(0002, 0010) Transfer Syntax UID                 UI: Implicit VR Little Endian
(0002, 0012) Implementation Class UID            UI: 1.3.6.1.4.1.9590.100.1.0.100.4.0
(0002, 0013) Implementation Version Name         SH: 'MATLAB IPT 4.0'
-------------------------------------------------
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.3.6.1.4.1.9590.100.1.1.167963314710984872700115334653369750841
(0008, 0020) Study Date    

In [24]:
import os

In [25]:
os.getcwd()

'C:\\Users\\lucasdevries\\surfdrive\\Projects\\ppinn'